In [6]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import dataloader_decoder
from torch.utils.data import DataLoader
from torch.nn import functional as F
import json
from decoder_models import *
import h5py
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
from tqdm import tqdm

In [22]:

# import argparse
# parser = argparse.ArgumentParser(description='calculate overlap of Decoder model')
# parser.add_argument('model_dir', type=str)
# args = parser.parse_args()
# model_dir = args.model_dir

model_dir = '/ltenas8/users/anneclaire/retrieval_202306/decoder/decoder_runs_20230628_newSpectra_wAS/decoderX/'

In [23]:

config = json.load(open(model_dir+'/config.json'))
model_name=config['model_name']
model = locals()[model_name](config['n_channels'], config['n_hidden_layers'],config['n_input_features'])#.to('cuda')
checkpoint = torch.load(config['directory']+'checkpoint.pth')#config['checkpoint_name'])
model.load_state_dict(checkpoint['model_state'])


<All keys matched successfully>

In [24]:
val_dataset = dataloader_decoder.decoder_dataset(config['input_ds_path'], config['syn_dataset_path'], config['target_freq'],
                                           i_start_ds = config['i_start_val'], i_end_ds = config['i_start_val']+config['val_size'],
                                           i_start_spec = config['i_start_spec'], i_end_spec = config['i_end_spec'],
                                           inds_input_features=config['inds_input_vars_'+config['target_freq']],

                                           normalize_input = config['normalize_input'],
                                           normalize_output = config['normalize_output'],

                                           mean_normalization_spec_json = config['normalize_path']+'/means_spec.json',
                                           std_normalization_spec_json = config['normalize_path']+'/stds_spec.json',

                                           mean_normalization_input_npy = config['normalize_path']+'/means_input.npy',
                                           std_normalization_input_npy = config['normalize_path']+'/stds_input.npy',

                                           shuffle_pre_train = config['shuffle_pre_train'])

N = len(val_dataset)

val_dataloader = DataLoader(val_dataset, num_workers=32, batch_size=1000, pin_memory=True)



In [25]:
mean_normalization_spec_json = config['normalize_path']+'/means_spec.json'
std_normalization_spec_json = config['normalize_path']+'/stds_spec.json'

mean_norm_X = json.load(open(mean_normalization_spec_json))['specX']
mean_norm_Ka = json.load(open(mean_normalization_spec_json))['specKa']
mean_norm_W = json.load(open(mean_normalization_spec_json))['specW']
std_norm_X = json.load(open(std_normalization_spec_json))['specX']
std_norm_Ka = json.load(open(std_normalization_spec_json))['specKa']
std_norm_W = json.load(open(std_normalization_spec_json))['specW']

if config['target_freq']=='W':
    mean_norm = mean_norm_W
    std_norm = std_norm_W
elif config['target_freq']=='Ka':
    mean_norm = mean_norm_Ka
    std_norm = std_norm_Ka
elif config['target_freq']=='X':
    mean_norm = mean_norm_X
    std_norm = std_norm_X
    


In [26]:
total_size=0
mean_rec=0
total_size2=0
mean_rec2=0
mean_rec3 = 0
total_size3 = 0
mean_rec4 = 0
total_size4 = 0
epochloss=0
with torch.no_grad():
    for (input_data, target) in tqdm(iter(val_dataloader)):
        output = model.decode(input_data)
        amplitude = target.max(axis=-1).values*std_norm-target.min(axis=-1).values*std_norm

        target_renorm = target*std_norm+mean_norm
        output_renorm = output[:,0,:]*std_norm+mean_norm
        target_remin = target_renorm -torch.min(target_renorm,axis=-1).values[:,None]
        output_remin = (output_renorm -torch.min(target_renorm,axis=-1).values[:,None]).abs()
        otmin = torch.min(target_remin,output_remin)
        otmax = torch.max(target_remin,output_remin)

        # compute first version of overlap on the current batch
        overlap3 = .5*(torch.sum(otmin,axis=-1)/torch.sum(output_remin,axis=-1) + torch.sum(otmin,axis=-1)/torch.sum(target_remin,axis=-1))
        # store in the mean_rec3 variable, keeping only spectra with amplitude > 4 (essentially to avoid spectra which are all noise)
        mean_rec3 += torch.mean(overlap3[amplitude>4])*output[amplitude>4].size(0)
        total_size3+=output[amplitude>4].size(0)

        # compute second version of overlap on the current batch
        overlap4 = torch.sum(otmin,axis=-1)/torch.sum(otmax,axis=-1)
        # store in the mean_rec4 variable, keeping only spectra with amplitude > 4
        mean_rec4 += torch.mean(overlap4[amplitude>4])*output[amplitude>4].size(0)
        total_size4+=output[amplitude>4].size(0)
        
        

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

In [27]:
mean_rec3/total_size3

tensor(0.9729)

In [ ]:
mean_rec3/total_size3

tensor(0.9813)

In [17]:
mean_rec3/total_size3

tensor(0.9693)

In [ ]:
        
with open(model_dir+'/overlap_metrics.txt','w') as f:
    f.write('recouvrement 3: %.3f\n'%(mean_rec3/total_size3))
    f.write('recouvrement 4: %.3f'%(mean_rec4/total_size4))
  